In [19]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pickle
import os
import json
import numpy as np


Firebase signin

In [20]:

cred = credentials.Certificate("../codeinterface-85b5e-firebase-adminsdk-11q7e-837ba92a03.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [21]:
docs = db.collection('tasks').get()
# save docs in a pickle file called old_tasks.pkl
tasks = {}
for doc in docs:
    doc_dict = doc.to_dict()
    doc_dict['id'] = doc.id 
    tasks[doc.id] = doc_dict
print(len(tasks))


385


In [22]:
today_date_string = str(np.datetime64('now'))
with open('./saved_data/old_tasks'+today_date_string+'.pkl', 'wb') as f:
    pickle.dump(tasks, f)

In [23]:
# remove all the documents from the collection
for doc in docs:
    doc.reference.delete()

In [24]:

folder_path = './tasks'  # Replace with the actual folder path

# Initialize a list to store all tasks
all_tasks = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            # Load JSON content
            task_data = json.load(file)
            # Add the task to the list
            all_tasks.append(task_data)

all_tasks = np.array(all_tasks)

In [7]:
len(all_tasks)

17

In [25]:
# for each task in all_tasks print name and type
for task in all_tasks:
    print(task['name'], "- Type:  ", task['type'])

table_transform_unnamed2 - Type:   data_manip
retriever - Type:   lengthy_code
tokenizer - Type:   edit_code
is_bored - Type:   logic
t_test - Type:   data_manip_math
event_scheduler - Type:   logic
order_by_points - Type:   logic
encode_message - Type:   logic
triple_sum_to_zero - Type:   logic
even_odd_count - Type:   logic
sum_product - Type:   logic
calculator - Type:   fix_code
table_transform_unnamed1 - Type:   data_manip
login_authenticator - Type:   lengthy_code
is_multiply_prime - Type:   logic
count_nums - Type:   logic
table_transform_named - Type:   data_manip


In [26]:
# find task in all_tasks with field name  is Average
def get_task_index(task_name):
    for i in range(len(all_tasks)):
        if all_tasks[i]['name'] == task_name:
            return i
    # raise error if task not found
    raise ValueError('Task not found')
tutorial_task_index = get_task_index('sum_product')

task_sets = [np.array([ get_task_index('even_odd_count'), get_task_index('triple_sum_to_zero'),   get_task_index('table_transform_named'), get_task_index('tokenizer'),
                          get_task_index('encode_message'), get_task_index('t_test'), get_task_index('event_scheduler')]),

            np.array([ get_task_index('even_odd_count'), get_task_index('is_bored'),   get_task_index('login_authenticator'), get_task_index('is_multiply_prime'),
                          get_task_index('count_nums'), get_task_index('table_transform_named'), get_task_index('calculator')]),

            np.array([ get_task_index('even_odd_count'), get_task_index('count_nums'),   get_task_index('calculator'), get_task_index('table_transform_unnamed2'),
                          get_task_index('login_authenticator'), get_task_index('encode_message'), get_task_index('is_bored')]),

            np.array([ get_task_index('even_odd_count'), get_task_index('order_by_points'),   get_task_index('retriever'), get_task_index('triple_sum_to_zero'),
                          get_task_index('tokenizer'), get_task_index('event_scheduler'), get_task_index('encode_message')]),

            np.array([ get_task_index('even_odd_count'), get_task_index('is_multiply_prime'),   get_task_index('table_transform_unnamed1'), get_task_index('t_test'),
                          get_task_index('is_bored'), get_task_index('order_by_points'), get_task_index('triple_sum_to_zero')])]           


In [27]:
no_model_tasks = []
autocomplete_gpt35_tasks = []
autocomplete_llama7_tasks = []
autocomplete_llama34_tasks = []
chat_gpt35_tasks = []
chat_llama7_tasks = []
chat_llama34_tasks = []
both_gpt35_tasks = []
tutorial_task = all_tasks[tutorial_task_index]
j = 0
for task_set in task_sets:
    selected_tasks = all_tasks[task_set]
    base_task = {
    'function_signatures': [task['function_signature'] for task in selected_tasks],
    'task_descriptions': [task['task_description'] for task in selected_tasks],
    'tutorial_function_signature': tutorial_task['function_signature'],
    'tutorial_task_description': tutorial_task['task_description'],
    'unit_tests': [task['unit_test'] for task in selected_tasks],
    'exp_condition': 'final_study',
    'tutorial_unit_test': tutorial_task['unit_test'],
    'task_completed': 0,}
    no_model_task = {**base_task,
                      'model': 'none',
                      'id': 'nomodel_'+str(j),
                      'max_tokens': 64,
                      'interface_type': 'autocomplete'
                      }
    autocomplete_gpt35_task = {**base_task,
                        'model': 'gpt35',
                        'id': 'autocomplete_gpt35_'+str(j),
                        'max_tokens': 64,
                        'interface_type': 'autocomplete'
                        }
    autocomplete_llama7_task = {**base_task,
                        'model': 'togethercomputer/CodeLlama-7b',
                        'id': 'autocomplete_llama7_'+str(j),
                        'max_tokens': 64,
                        'interface_type': 'autocomplete'
                        }
    autocomplete_llama34_task = {**base_task,
                        'model': 'togethercomputer/CodeLlama-34b',
                        'id': 'autocomplete_llama34_'+str(j),
                        'max_tokens': 64,
                        'interface_type': 'autocomplete'
                        }
    chat_gpt35_task = {**base_task,
                        'model': 'gpt-3.5-turbo',
                        'id': 'chat_gpt35_'+str(j),
                        'max_tokens': 512,
                        'interface_type': 'chat'
                        }
    chat_llama7_task = {**base_task,
                        'model': 'CodeLlama-7b-Instruct',
                        'id': 'chat_llama7_'+str(j),
                        'max_tokens': 512,
                        'interface_type': 'chat'
                        }
    chat_llama34_task = {**base_task,
                        'model': 'CodeLlama-34b-Instruct',
                        'id': 'chat_llama34_'+str(j),
                        'max_tokens': 512,
                        'interface_type': 'chat'
                        }
    both_gpt35_task = {**base_task,
                        'model': 'gpt-3.5-turbo', # gpt-3.5-turbo
                        'id': 'both_gpt35_'+str(j),
                        'max_tokens': 512,
                        'interface_type': 'both'
                        }

    j+=1
    no_model_tasks.append(no_model_task)
    autocomplete_gpt35_tasks.append(autocomplete_gpt35_task)
    autocomplete_llama7_tasks.append(autocomplete_llama7_task)
    autocomplete_llama34_tasks.append(autocomplete_llama34_task)
    chat_gpt35_tasks.append(chat_gpt35_task)
    chat_llama7_tasks.append(chat_llama7_task)
    chat_llama34_tasks.append(chat_llama34_task)
    both_gpt35_tasks.append(both_gpt35_task)



In [28]:
for i in range(11):
    for j in range(len(task_sets)):
        db.collection('tasks').document(no_model_tasks[j]['id']+'_'+str(i)).set(no_model_tasks[j])
        db.collection('tasks').document(autocomplete_gpt35_tasks[j]['id']+'_'+str(i)).set(autocomplete_gpt35_tasks[j])
        db.collection('tasks').document(autocomplete_llama7_tasks[j]['id']+'_'+str(i)).set(autocomplete_llama7_tasks[j])
        db.collection('tasks').document(autocomplete_llama34_tasks[j]['id']+'_'+str(i)).set(autocomplete_llama34_tasks[j])
        db.collection('tasks').document(chat_gpt35_tasks[j]['id']+'_'+str(i)).set(chat_gpt35_tasks[j])
        db.collection('tasks').document(chat_llama7_tasks[j]['id']+'_'+str(i)).set(chat_llama7_tasks[j])
        db.collection('tasks').document(chat_llama34_tasks[j]['id']+'_'+str(i)).set(chat_llama34_tasks[j])
        #db.collection('tasks').document(both_gpt35_tasks[j]['id']+'_'+str(i)).set(both_gpt35_tasks[j])
        
